## 1. Configuração do Ambiente

In [ ]:
# ============================================================================
# CONFIGURAÇÃO DO AMBIENTE
# ============================================================================

import warnings
warnings.filterwarnings('ignore')

# Bibliotecas padrão
import json
from pathlib import Path
from datetime import datetime

# Computação científica
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import linregress, t as t_dist

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 150,
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'text.usetex': False,  # Usar False para compatibilidade
})

# Caminhos do projeto
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUTS_DIR = DATA_DIR / 'outputs'
FIGURES_DIR = OUTPUTS_DIR / 'figures'
TABLES_DIR = OUTPUTS_DIR / 'tables'

# Criar diretórios se não existirem
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print("="*60)
print("FASE 3: ANÁLISE DE RISCO SISTEMÁTICO (CAPM)")
print("="*60)
print(f"\nData de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Diretório do projeto: {PROJECT_ROOT}")

## 2. Carregamento dos Dados

In [ ]:
# ============================================================================
# CARREGAMENTO DOS DADOS
# ============================================================================

# Carregar retornos processados
returns_path = PROCESSED_DIR / 'returns.csv'
df_returns = pd.read_csv(returns_path, parse_dates=['date'])
df_returns.set_index('date', inplace=True)

# Carregar SELIC
selic_path = PROCESSED_DIR / 'selic.csv'
df_selic = pd.read_csv(selic_path, parse_dates=['date'])

print("\n" + "="*60)
print("DADOS CARREGADOS")
print("="*60)

print(f"\n📊 Retornos (returns.csv):")
print(f"   • Período: {df_returns.index.min().strftime('%Y-%m-%d')} a {df_returns.index.max().strftime('%Y-%m-%d')}")
print(f"   • Observações: {len(df_returns):,}")
print(f"   • Colunas: {list(df_returns.columns)}")

print(f"\n💰 Taxa SELIC (selic.csv):")
print(f"   • Registros: {len(df_selic):,}")
print(f"   • SELIC média (diária): {df_selic['selic_daily'].mean()*100:.4f}%")
print(f"   • SELIC anualizada (média): {df_selic['selic_annual'].mean()*100:.2f}%")

# Preview dos dados
print("\n📋 Preview dos retornos:")
df_returns.head()

## 3. Regressão OLS — Estimação do Beta

O modelo de mercado é estimado via regressão linear:

$$r_{PETR4,t} - r_{f,t} = \alpha + \beta \cdot (r_{IBOV,t} - r_{f,t}) + \epsilon_t$$

Onde:
- $r_{PETR4,t} - r_{f,t}$: Retorno excedente do ativo
- $r_{IBOV,t} - r_{f,t}$: Retorno excedente do mercado
- $\alpha$: Intercepto (Jensen's Alpha)
- $\beta$: Coeficiente de risco sistemático
- $\epsilon_t$: Termo de erro (risco idiossincrático)

In [ ]:
# ============================================================================
# REGRESSÃO OLS — ESTIMAÇÃO DO BETA
# ============================================================================

# Variáveis para regressão (retornos excedentes)
X = df_returns['r_ibov_excess'].values  # Variável independente (mercado)
Y = df_returns['r_petr4_excess'].values  # Variável dependente (PETR4)

# Regressão linear via scipy
slope, intercept, r_value, p_value, std_err = linregress(X, Y)

# Parâmetros estimados
beta = slope
alpha = intercept
r_squared = r_value ** 2
n = len(X)

# R² Ajustado
k = 1  # número de variáveis independentes
r_squared_adj = 1 - (1 - r_squared) * (n - 1) / (n - k - 1)

# Estatística t para o beta
t_stat_beta = beta / std_err

# Intervalo de confiança 95% para o beta
df_residual = n - 2  # graus de liberdade
t_critical = t_dist.ppf(0.975, df_residual)  # bilateral
ci_lower = beta - t_critical * std_err
ci_upper = beta + t_critical * std_err

# Cálculo do erro padrão do alfa
y_pred = alpha + beta * X
residuals = Y - y_pred
mse = np.sum(residuals**2) / df_residual
se_alpha = np.sqrt(mse * (1/n + np.mean(X)**2 / np.sum((X - np.mean(X))**2)))
t_stat_alpha = alpha / se_alpha
p_value_alpha = 2 * (1 - t_dist.cdf(abs(t_stat_alpha), df_residual))

# Volatilidade idiossincrática (anualizada)
idio_vol_daily = np.std(residuals)
idio_vol_annual = idio_vol_daily * np.sqrt(252)

print("\n" + "="*60)
print("RESULTADOS DA REGRESSÃO OLS")
print("="*60)
print(f"\nModelo: r_PETR4 - rf = α + β × (r_IBOV - rf) + ε")
print(f"\n{'─'*60}")
print(f"{'Parâmetro':<25} {'Valor':>15} {'Erro Padrão':>12} {'t-stat':>10}")
print(f"{'─'*60}")
print(f"{'Beta (β)':<25} {beta:>15.4f} {std_err:>12.4f} {t_stat_beta:>10.2f}")
print(f"{'Alfa (α) diário':<25} {alpha:>15.6f} {se_alpha:>12.6f} {t_stat_alpha:>10.2f}")
print(f"{'─'*60}")

print(f"\n📊 Estatísticas de Ajuste:")
print(f"   • R²: {r_squared:.4f} ({r_squared*100:.2f}%)")
print(f"   • R² Ajustado: {r_squared_adj:.4f} ({r_squared_adj*100:.2f}%)")
print(f"   • Observações (n): {n:,}")
print(f"   • Graus de liberdade: {df_residual:,}")

print(f"\n🎯 Beta — Intervalo de Confiança 95%:")
print(f"   • IC: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"   • p-valor: {p_value:.2e} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''}")

print(f"\n📈 Alfa (Jensen's Alpha):")
print(f"   • Alfa diário: {alpha*100:.4f}%")
print(f"   • Alfa anualizado: {alpha*252*100:.2f}%")
print(f"   • p-valor: {p_value_alpha:.4f} {'***' if p_value_alpha < 0.001 else '**' if p_value_alpha < 0.01 else '*' if p_value_alpha < 0.05 else 'NS'}")

print(f"\n⚡ Risco Idiossincrático:")
print(f"   • Volatilidade diária: {idio_vol_daily*100:.4f}%")
print(f"   • Volatilidade anual: {idio_vol_annual*100:.2f}%")

## 4. Ajuste de Blume

O ajuste de Blume (1971) corrige o viés de estimação do beta histórico para uso prospectivo:

$$\beta_{adj} = 0.67 \times \hat{\beta} + 0.33 \times 1.0$$

Este ajuste reflete a tendência empírica dos betas a reverterem para a média de mercado (1.0) ao longo do tempo.

In [ ]:
# ============================================================================
# AJUSTE DE BLUME
# ============================================================================

# Coeficientes do ajuste de Blume
BLUME_WEIGHT = 0.67
MEAN_BETA = 1.0

# Beta ajustado
beta_blume = BLUME_WEIGHT * beta + (1 - BLUME_WEIGHT) * MEAN_BETA

print("\n" + "="*60)
print("AJUSTE DE BLUME")
print("="*60)
print(f"\nFórmula: β_adj = {BLUME_WEIGHT} × β_raw + {1-BLUME_WEIGHT} × {MEAN_BETA}")
print(f"\n{'─'*40}")
print(f"{'Beta Estimado (raw)':<25} {beta:>10.4f}")
print(f"{'Beta Ajustado (Blume)':<25} {beta_blume:>10.4f}")
print(f"{'Diferença':<25} {beta_blume - beta:>10.4f}")
print(f"{'─'*40}")

print(f"\n💡 Interpretação:")
print(f"   O beta ajustado de {beta_blume:.4f} é {'menor' if beta_blume < beta else 'maior'} que o beta estimado.")
print(f"   Isso reflete a reversão à média esperada para betas {'acima' if beta > 1 else 'abaixo'} de 1.0.")

## 5. Custo de Capital Próprio (Ke)

O custo de capital próprio é estimado via equação do CAPM:

$$K_e = R_f + \beta \times ERP$$

Onde:
- $R_f$: Taxa livre de risco (SELIC)
- $ERP$: Equity Risk Premium (Prêmio de Risco de Mercado)

In [ ]:
# ============================================================================
# CUSTO DE CAPITAL PRÓPRIO (Ke)
# ============================================================================

# Parâmetros
ERP_BRAZIL = 0.055  # Equity Risk Premium para Brasil (5.5%)

# Taxa livre de risco (última SELIC anualizada)
rf_annual = df_selic['selic_annual'].iloc[-1]
rf_mean = df_selic['selic_annual'].mean()

# Retornos anualizados observados
r_petr4_annual = df_returns['r_petr4'].mean() * 252
r_ibov_annual = df_returns['r_ibov'].mean() * 252

# Prêmio de mercado realizado
market_premium_realized = r_ibov_annual - rf_mean

# Custo de capital próprio (usando beta raw e ajustado)
ke_raw = rf_mean + beta * ERP_BRAZIL
ke_blume = rf_mean + beta_blume * ERP_BRAZIL

# Ke usando prêmio de mercado realizado
ke_realized = rf_mean + beta * market_premium_realized

# Mispricing (alfa realizado vs. esperado)
expected_return = ke_raw
realized_return = r_petr4_annual
mispricing = realized_return - expected_return

print("\n" + "="*60)
print("CUSTO DE CAPITAL PRÓPRIO (Ke)")
print("="*60)

print(f"\n📊 Parâmetros de Entrada:")
print(f"   • Taxa Livre de Risco (Rf média): {rf_mean*100:.2f}%")
print(f"   • Taxa Livre de Risco (Rf atual): {rf_annual*100:.2f}%")
print(f"   • ERP Brasil (assumido): {ERP_BRAZIL*100:.1f}%")
print(f"   • Prêmio de Mercado (realizado): {market_premium_realized*100:.2f}%")

print(f"\n{'─'*50}")
print(f"{'Métrica':<35} {'Valor':>12}")
print(f"{'─'*50}")
print(f"{'Ke (β raw, ERP 5.5%)':<35} {ke_raw*100:>11.2f}%")
print(f"{'Ke (β Blume, ERP 5.5%)':<35} {ke_blume*100:>11.2f}%")
print(f"{'Ke (β raw, prêmio realizado)':<35} {ke_realized*100:>11.2f}%")
print(f"{'─'*50}")
print(f"{'Retorno Realizado PETR4':<35} {realized_return*100:>11.2f}%")
print(f"{'Retorno Realizado IBOV':<35} {r_ibov_annual*100:>11.2f}%")
print(f"{'─'*50}")

print(f"\n🎯 Análise de Mispricing:")
print(f"   • Retorno Esperado (CAPM): {expected_return*100:.2f}%")
print(f"   • Retorno Realizado: {realized_return*100:.2f}%")
print(f"   • Mispricing (α): {mispricing*100:+.2f}%")

if mispricing > 0:
    print(f"\n   ✅ PETR4 gerou retorno ACIMA do esperado pelo CAPM (α > 0)")
    print(f"      Interpretação: O ativo pode ter sido SUBVALORIZADO pelo mercado.")
else:
    print(f"\n   ⚠️ PETR4 gerou retorno ABAIXO do esperado pelo CAPM (α < 0)")
    print(f"      Interpretação: O ativo pode ter sido SOBREVALORIZADO pelo mercado.")

## 6. Persistência dos Resultados

In [ ]:
# ============================================================================
# PERSISTÊNCIA DOS RESULTADOS
# ============================================================================

# Estrutura completa dos resultados
capm_results = {
    "model": {
        "name": "CAPM - Capital Asset Pricing Model",
        "dependent_variable": "r_petr4_excess",
        "independent_variable": "r_ibov_excess",
        "equation": "r_PETR4 - rf = α + β × (r_IBOV - rf) + ε"
    },
    "sample": {
        "period_start": df_returns.index.min().strftime('%Y-%m-%d'),
        "period_end": df_returns.index.max().strftime('%Y-%m-%d'),
        "n_observations": int(n),
        "frequency": "daily"
    },
    "coefficients": {
        "beta": {
            "estimate": round(beta, 6),
            "std_error": round(std_err, 6),
            "t_statistic": round(t_stat_beta, 4),
            "p_value": round(p_value, 10),
            "ci_95_lower": round(ci_lower, 6),
            "ci_95_upper": round(ci_upper, 6),
            "significant": p_value < 0.05
        },
        "alpha": {
            "estimate_daily": round(alpha, 8),
            "estimate_annual": round(alpha * 252, 6),
            "std_error": round(se_alpha, 8),
            "t_statistic": round(t_stat_alpha, 4),
            "p_value": round(p_value_alpha, 6),
            "significant": p_value_alpha < 0.05
        },
        "beta_blume": {
            "estimate": round(beta_blume, 6),
            "adjustment_formula": "0.67 × β_raw + 0.33 × 1.0"
        }
    },
    "fit_statistics": {
        "r_squared": round(r_squared, 6),
        "r_squared_adjusted": round(r_squared_adj, 6),
        "correlation": round(r_value, 6),
        "residual_std_daily": round(idio_vol_daily, 6),
        "residual_std_annual": round(idio_vol_annual, 6),
        "degrees_of_freedom": int(df_residual)
    },
    "cost_of_equity": {
        "rf_annual": round(rf_mean, 6),
        "rf_current": round(rf_annual, 6),
        "erp_assumed": ERP_BRAZIL,
        "market_premium_realized": round(market_premium_realized, 6),
        "ke_raw": round(ke_raw, 6),
        "ke_blume": round(ke_blume, 6),
        "ke_realized_premium": round(ke_realized, 6)
    },
    "returns": {
        "petr4_annual": round(r_petr4_annual, 6),
        "ibov_annual": round(r_ibov_annual, 6),
        "expected_capm": round(expected_return, 6),
        "mispricing": round(mispricing, 6)
    },
    "metadata": {
        "generated_at": datetime.now().isoformat(),
        "notebook": "02_capm_analysis.ipynb",
        "phase": "Fase 3 - Análise de Risco Sistemático"
    }
}

# Salvar JSON
output_path = PROCESSED_DIR / 'capm_regression.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(capm_results, f, indent=2, ensure_ascii=False)

print("\n" + "="*60)
print("RESULTADOS PERSISTIDOS")
print("="*60)
print(f"\n✅ Arquivo salvo: {output_path}")
print(f"\n📋 Estrutura do JSON:")
for key in capm_results.keys():
    print(f"   • {key}")

## 7. Geração da Tabela LaTeX

In [ ]:
# ============================================================================
# GERAÇÃO DA TABELA LaTeX
# ============================================================================

def format_pvalue(p):
    """Formata p-valor com asteriscos de significância."""
    if p < 0.001:
        return f"{p:.2e}***"
    elif p < 0.01:
        return f"{p:.4f}**"
    elif p < 0.05:
        return f"{p:.4f}*"
    else:
        return f"{p:.4f}"

latex_table = rf"""\begin{{table}}[htbp]
\centering
\caption{{Resultados da Estimação CAPM — PETR4}}
\label{{tab:capm_results}}
\begin{{tabular}}{{lcc}}
\toprule
\textbf{{Estatística}} & \textbf{{Valor}} & \textbf{{Erro Padrão}} \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Coeficientes}}}} \\
\quad Beta ($\beta$) & {beta:.4f}*** & ({std_err:.4f}) \\
\quad IC 95\% & [{ci_lower:.4f}, {ci_upper:.4f}] & — \\
\quad Alfa ($\alpha$) anualizado & {alpha*252*100:.2f}\% & ({se_alpha*252*100:.2f}\%) \\
\quad Beta Ajustado (Blume) & {beta_blume:.4f} & — \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Estatísticas de Ajuste}}}} \\
\quad $R^2$ & {r_squared:.4f} & — \\
\quad $R^2$ Ajustado & {r_squared_adj:.4f} & — \\
\quad Volatilidade Idiossincrática & {idio_vol_annual*100:.2f}\% & — \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Custo de Capital}}}} \\
\quad Taxa Livre de Risco ($R_f$) & {rf_mean*100:.2f}\% & — \\
\quad Prêmio de Risco (ERP) & {ERP_BRAZIL*100:.1f}\% & — \\
\quad $K_e$ (CAPM) & {ke_raw*100:.2f}\% & — \\
\quad $K_e$ (Blume) & {ke_blume*100:.2f}\% & — \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Performance}}}} \\
\quad Retorno Realizado & {realized_return*100:.2f}\% & — \\
\quad Mispricing ($\alpha$) & {mispricing*100:+.2f}\% & — \\
\bottomrule
\end{{tabular}}

\vspace{{0.3cm}}
\footnotesize
\textit{{Nota}}: Período de análise: {df_returns.index.min().strftime('%Y-%m-%d')} a {df_returns.index.max().strftime('%Y-%m-%d')} (n = {n:,}). \\
*** p < 0.001, ** p < 0.01, * p < 0.05.
\end{{table}}
"""

# Salvar tabela
table_path = TABLES_DIR / 'resultados_capm.tex'
with open(table_path, 'w', encoding='utf-8') as f:
    f.write(latex_table)

print("\n" + "="*60)
print("TABELA LaTeX GERADA")
print("="*60)
print(f"\n✅ Arquivo salvo: {table_path}")
print(f"\n📋 Preview:")
print(latex_table[:500] + "...")

## 8. Figura: Regressão Beta (Scatter Plot)

In [ ]:
# ============================================================================
# FIGURA: REGRESSÃO BETA (SCATTER PLOT)
# ============================================================================

fig, ax = plt.subplots(figsize=(10, 7))

# Scatter plot dos retornos
ax.scatter(X * 100, Y * 100, alpha=0.3, s=20, c='steelblue', edgecolors='none',
           label=f'Observações (n={n:,})')

# Linha de regressão
x_line = np.linspace(X.min(), X.max(), 100)
y_line = alpha + beta * x_line
ax.plot(x_line * 100, y_line * 100, 'r-', linewidth=2.5, 
        label=f'Linha de Regressão: β = {beta:.4f}')

# Linha de 45° (beta = 1)
ax.plot(x_line * 100, x_line * 100, 'k--', linewidth=1, alpha=0.5,
        label='β = 1 (mercado)')

# Bandas de confiança (aproximado)
se_pred = np.sqrt(mse * (1/n + (x_line - np.mean(X))**2 / np.sum((X - np.mean(X))**2)))
y_upper = (alpha + beta * x_line + t_critical * se_pred) * 100
y_lower = (alpha + beta * x_line - t_critical * se_pred) * 100
ax.fill_between(x_line * 100, y_lower, y_upper, alpha=0.15, color='red',
                label='IC 95%')

# Formatação
ax.set_xlabel('Retorno Excedente do Mercado (IBOV) — %', fontsize=12)
ax.set_ylabel('Retorno Excedente de PETR4 — %', fontsize=12)
ax.set_title('Regressão CAPM: Estimação do Beta\nPETR4 vs. IBOVESPA', fontsize=14, fontweight='bold')

# Anotação com estatísticas
stats_text = f"""β = {beta:.4f} ± {std_err:.4f}
R² = {r_squared:.4f}
α = {alpha*252*100:.2f}% a.a.
n = {n:,}"""
ax.annotate(stats_text, xy=(0.05, 0.95), xycoords='axes fraction',
            fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.legend(loc='lower right', fontsize=9)
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5, alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5, alpha=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()

# Salvar figura
fig_path = FIGURES_DIR / 'regressao_beta.pdf'
fig.savefig(fig_path, format='pdf', bbox_inches='tight', dpi=300)

print(f"\n✅ Figura salva: {fig_path}")
plt.show()

## 9. Figura: Security Market Line (SML)

In [ ]:
# ============================================================================
# FIGURA: SECURITY MARKET LINE (SML)
# ============================================================================

fig, ax = plt.subplots(figsize=(10, 7))

# Parâmetros da SML
betas_sml = np.linspace(0, 2.5, 100)
returns_sml = rf_mean + betas_sml * ERP_BRAZIL

# Linha SML
ax.plot(betas_sml, returns_sml * 100, 'b-', linewidth=2.5, 
        label=f'SML: E(R) = {rf_mean*100:.1f}% + β × {ERP_BRAZIL*100:.1f}%')

# Ponto do mercado (beta = 1)
market_return = rf_mean + 1 * ERP_BRAZIL
ax.scatter([1], [market_return * 100], s=150, c='green', marker='D', 
           zorder=5, label=f'Mercado (β=1): {market_return*100:.1f}%')

# Ponto de PETR4 (retorno esperado CAPM)
ax.scatter([beta], [ke_raw * 100], s=200, c='red', marker='s', 
           zorder=5, edgecolors='darkred', linewidths=2,
           label=f'PETR4 Esperado (β={beta:.2f}): {ke_raw*100:.1f}%')

# Ponto de PETR4 (retorno realizado)
ax.scatter([beta], [realized_return * 100], s=200, c='gold', marker='*', 
           zorder=6, edgecolors='orange', linewidths=2,
           label=f'PETR4 Realizado: {realized_return*100:.1f}%')

# Linha conectando esperado e realizado (Jensen's Alpha)
ax.annotate('', xy=(beta, realized_return * 100), 
            xytext=(beta, ke_raw * 100),
            arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
ax.annotate(f'α = {mispricing*100:+.2f}%', 
            xy=(beta + 0.08, (ke_raw + mispricing/2) * 100),
            fontsize=11, color='purple', fontweight='bold')

# Ponto da taxa livre de risco
ax.scatter([0], [rf_mean * 100], s=120, c='navy', marker='o', 
           zorder=5, label=f'Rf (SELIC): {rf_mean*100:.1f}%')

# Formatação
ax.set_xlabel('Beta (β)', fontsize=12)
ax.set_ylabel('Retorno Esperado — % a.a.', fontsize=12)
ax.set_title('Security Market Line (SML)\nPosicionamento de PETR4', fontsize=14, fontweight='bold')

ax.axhline(y=rf_mean * 100, color='navy', linestyle=':', linewidth=1, alpha=0.5)
ax.axvline(x=1, color='green', linestyle=':', linewidth=1, alpha=0.5)

ax.set_xlim(-0.1, 2.2)
ax.set_ylim(rf_mean * 100 - 2, max(realized_return * 100, ke_raw * 100) + 3)

ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()

# Salvar figura
fig_path = FIGURES_DIR / 'sml_capm.pdf'
fig.savefig(fig_path, format='pdf', bbox_inches='tight', dpi=300)

print(f"\n✅ Figura salva: {fig_path}")
plt.show()

## 10. Sumário e Conclusões

In [ ]:
# ============================================================================
# SUMÁRIO FINAL
# ============================================================================

print("\n" + "="*70)
print("SUMÁRIO — ANÁLISE CAPM DE PETR4")
print("="*70)

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│                    RESULTADOS PRINCIPAIS                            │
├─────────────────────────────────────────────────────────────────────┤
│  Período de Análise: {df_returns.index.min().strftime('%Y-%m-%d')} a {df_returns.index.max().strftime('%Y-%m-%d')}                   │
│  Observações: {n:,} dias de negociação                            │
├─────────────────────────────────────────────────────────────────────┤
│  RISCO SISTEMÁTICO                                                  │
│    • Beta Estimado (β):           {beta:>8.4f}                           │
│    • Beta Ajustado (Blume):       {beta_blume:>8.4f}                           │
│    • R² (poder explicativo):      {r_squared*100:>7.2f}%                           │
│    • Correlação PETR4 x IBOV:     {r_value:>8.4f}                           │
├─────────────────────────────────────────────────────────────────────┤
│  CUSTO DE CAPITAL                                                   │
│    • Taxa Livre de Risco (Rf):    {rf_mean*100:>7.2f}%                           │
│    • Prêmio de Risco (ERP):       {ERP_BRAZIL*100:>7.1f}%                           │
│    • Ke (CAPM):                   {ke_raw*100:>7.2f}%                           │
├─────────────────────────────────────────────────────────────────────┤
│  PERFORMANCE                                                        │
│    • Retorno Esperado (CAPM):     {expected_return*100:>7.2f}%                           │
│    • Retorno Realizado:           {realized_return*100:>7.2f}%                           │
│    • Jensen's Alpha (α):          {mispricing*100:>+7.2f}%                           │
├─────────────────────────────────────────────────────────────────────┤
│  INTERPRETAÇÃO                                                      │
│    {'✅ PETR4 apresentou alfa POSITIVO — superou o CAPM' if mispricing > 0 else '⚠️ PETR4 apresentou alfa NEGATIVO — abaixo do CAPM'}             │
│    Beta > 1 indica ativo MAIS VOLÁTIL que o mercado                 │
│    R² de {r_squared*100:.1f}% indica que {100-r_squared*100:.1f}% do risco é idiossincrático            │
└─────────────────────────────────────────────────────────────────────┘
""")

print("\n📁 Artefatos Gerados:")
print(f"   • {PROCESSED_DIR / 'capm_regression.json'}")
print(f"   • {TABLES_DIR / 'resultados_capm.tex'}")
print(f"   • {FIGURES_DIR / 'regressao_beta.pdf'}")
print(f"   • {FIGURES_DIR / 'sml_capm.pdf'}")

print("\n" + "="*70)
print("FASE 3 CONCLUÍDA COM SUCESSO")
print("="*70)